<a href="https://colab.research.google.com/github/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting/blob/main/notebooks/02_linear_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ARIMA-Based Baseline Model

### Notebook Setup

The following cells initialize the notebook's environment. This includes installing necessary libraries, mounting Google Drive for credentials, and downloading the competition dataset from Kaggle.

This section is **not part of the core modeling logic** and can be adapted based on your specific environment or data access methods.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q wandb kaggle onnx pandas numpy xgboost scikit-learn dagshub mlflow neuralforecast statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
! mkdir ~/.kaggle

!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c walmart-recruiting-store-sales-forecasting

! unzip -o walmart-recruiting-store-sales-forecasting.zip
! rm walmart-recruiting-store-sales-forecasting.zip
! unzip -oq '*.zip'

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 223MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           

4 archives were successfully processed.


In [4]:
from sklearn import set_config
# Set scikit-learn to output pandas DataFrames
set_config(transform_output="pandas")

In [5]:
# Initialize Weights & Biases for experiment tracking
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lchik22 (lchik22-free-uni) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


### Load and Split Data

This section defines helper functions to load the raw data, perform initial merging of `stores` and `features` tables, and split the data into training and validation sets based on a date cutoff.

In [6]:
import pandas as pd
import numpy as np
import logging
import torch
from itertools import product
from typing import Dict, List, Optional

In [7]:
def process_dates_and_sort(df: pd.DataFrame) -> pd.DataFrame:
    """Converts 'Date' column to datetime and sorts the DataFrame."""
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"])
        sort_keys = [col for col in ["Date", 'Store', 'Dept'] if col in df.columns]
        if sort_keys:
            df = df.sort_values(by=sort_keys).reset_index(drop=True)
    return df

def _merge_features(df: pd.DataFrame, features_df: pd.DataFrame) -> pd.DataFrame:
    """Helper to merge features data."""
    return pd.merge(df, features_df, on=['Store', 'Date', 'IsHoliday'], how='left')

def _merge_stores(df: pd.DataFrame, stores_df: pd.DataFrame) -> pd.DataFrame:
    """Helper to merge stores data."""
    return pd.merge(df, stores_df, on=['Store'], how='left')

In [8]:
def run_preprocessing(
    dataframes: Dict[str, pd.DataFrame],
    process_train: bool = True,
    process_test: bool = True,
    merge_features: bool = True,
    merge_stores: bool = True,
    drop_raw_components: bool = False
) -> Dict[str, pd.DataFrame]:
    """Orchestrates the initial merging and processing of the raw dataframes."""
    primary_to_process = []
    if process_train and "train" in dataframes:
        primary_to_process.append("train")
    if process_test and "test" in dataframes:
        primary_to_process.append("test")

    if not primary_to_process:
        return {}

    processed_dfs = {}

    for name in primary_to_process:
        df = dataframes[name].copy()

        if merge_features and "features" in dataframes:
            df = _merge_features(df, dataframes["features"])
        if merge_stores and "stores" in dataframes:
            df = _merge_stores(df, dataframes["stores"])

        df = process_dates_and_sort(df)

        processed_dfs[name] = df

    if drop_raw_components:
        keys_to_drop = primary_to_process
        if merge_features:
             keys_to_drop.append("features")
        if merge_stores:
             keys_to_drop.append("stores")

        for key in keys_to_drop:
            if key in dataframes:
                del dataframes[key]

    return processed_dfs

In [9]:
def split_data(
    dataframe: pd.DataFrame,
    separate_target: bool = True,
    target_column: str = "Weekly_Sales"
):
    """Splits the data into training and validation sets using a fixed date cutoff."""
    train_df = dataframe[dataframe["Date"] < pd.Timestamp('2011-11-30') ]
    valid_df = dataframe[dataframe["Date"] >= pd.Timestamp('2011-11-30') ]

    if separate_target:
        X_train = train_df.drop(columns=[target_column])
        y_train = train_df[target_column]
        X_valid = valid_df.drop(columns=[target_column])
        y_valid = valid_df[target_column]
        return X_train, y_train, X_valid, y_valid
    return train_df, valid_df


In [10]:
def load_raw_data(
    dataframes_to_load: Optional[List[str]] = None
) -> Dict[str, pd.DataFrame]:
    """Loads specified raw CSV files into a dictionary of pandas DataFrames."""
    AVAILABLE_DATAFRAMES = {
        "stores": '/content/stores.csv',
        "features": '/content/features.csv',
        "train": '/content/train.csv',
        "test": '/content/test.csv'
    }

    if dataframes_to_load is None:
        dataframes_to_load = list(AVAILABLE_DATAFRAMES.keys())
    else:
        for name in dataframes_to_load:
            if name not in AVAILABLE_DATAFRAMES:
                raise ValueError(
                    f"'{name}' is not a valid dataframe name. "
                    f"Choose from: {list(AVAILABLE_DATAFRAMES.keys())}"
                )
    loaded_dataframes = {}
    for name in dataframes_to_load:
        path = AVAILABLE_DATAFRAMES[name]
        loaded_dataframes[name] = pd.read_csv(path)

    return loaded_dataframes

In [11]:
dataframes = load_raw_data()
df = run_preprocessing(dataframes, process_test=False)['train']
X_train, y_train, X_valid, y_valid = split_data(df, separate_target=True)

print(f"Shapes of X_train and y_train: {X_train.shape}, {y_train.shape}")
print(f"Shapes of X_valid and y_valid: {X_valid.shape}, {y_valid.shape}")

Shapes of X_train and y_train: (279085, 15), (279085,)
Shapes of X_valid and y_valid: (142485, 15), (142485,)


# Data Cleaning and Feature Engineering

In this section, we apply several preprocessing steps to prepare the data for modeling. The key transformations are organized into a `scikit-learn` pipeline for reproducibility.

The feature engineering steps include:

- **Data Subsetting**: Training time-series models on every store-department combination is computationally expensive. We select a random sample of 100 (Store, Dept) pairs to create a representative subset, reducing training time while preserving general data patterns.

- **Dropping Uninformative Features**: The `MarkDown` columns contain a high percentage of missing values and are dropped to simplify the feature space.

- **Encoding Categorical Variables**: We apply one-hot encoding to the `Type` feature and convert the boolean `IsHoliday` column to integers (0/1) to make them suitable for linear models.

- **Adding Time-Based Features**: We extract cyclical and event-based features from the `Date` column, such as `Year`, `Month`, `WeekOfYear`, Fourier terms (sine/cosine transformations), and proximity to holidays. These features are designed to help time-series models like SARIMAX capture seasonality and holiday effects.

To accelerate model training, we will work with a random subset of 100 store-department combinations.

In [12]:
import pandas as pd

# Create temporary copies to avoid modifying the original dataframes in place
X_train_ = X_train.reset_index(drop=True)
X_valid_ = X_valid.reset_index(drop=True)
y_train_ = y_train.reset_index(drop=True)
y_valid_ = y_valid.reset_index(drop=True)

# Select 100 unique (Store, Dept) pairs to form the subset
subset_keys = X_train_[['Store', 'Dept']].drop_duplicates().sample(n=100, random_state=42)

# Filter the training and validation sets based on the selected keys
X_train = pd.merge(X_train_, subset_keys, on=['Store', 'Dept'], how='inner')
X_valid = pd.merge(X_valid_, subset_keys, on=['Store', 'Dept'], how='inner')

# Align the target variables (y_train, y_valid) with the new filtered feature sets
y_train = y_train_.iloc[X_train.index]
y_valid = y_valid_.iloc[X_valid.index]

X_train

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,19,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,1,32,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
2,1,59,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
3,1,95,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
4,2,8,2010-02-05,False,40.19,2.572,NaN,NaN,NaN,NaN,NaN,210.752605,8.324,A,202307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8239,44,5,2011-11-25,True,38.89,3.445,NaN,NaN,NaN,NaN,256.45,129.836400,6.078,C,39910
8240,44,95,2011-11-25,True,38.89,3.445,NaN,NaN,NaN,NaN,256.45,129.836400,6.078,C,39910
8241,45,27,2011-11-25,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,188.350400,8.523,B,118221
8242,45,36,2011-11-25,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,188.350400,8.523,B,118221


In [13]:
from sklearn.base import BaseEstimator, TransformerMixin

class ChangeColumns(BaseEstimator, TransformerMixin):
    """A custom transformer to select or drop columns from a DataFrame."""
    def __init__(self, columns_to_drop=None, columns_to_keep=None):
        self.columns_to_drop = columns_to_drop
        self.columns_to_keep = columns_to_keep

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_copy = X.copy()

        if self.columns_to_keep is not None:
            return X_copy[self.columns_to_keep]

        if self.columns_to_drop is not None:
            return X_copy.drop(columns=self.columns_to_drop, errors='ignore')

        return X_copy

First, we drop the `MarkDown` columns. These columns have a large number of missing values and were introduced late in the dataset's timeline, making them unreliable for a general-purpose model.

In [14]:
columns_to_drop=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
drop_markdowns = ChangeColumns(columns_to_drop=columns_to_drop)
X_train_t = drop_markdowns.fit_transform(X_train)
X_valid_t = drop_markdowns.transform(X_valid)

X_train_t

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Type,Size
0,1,19,2010-02-05,False,42.31,2.572,211.096358,8.106,A,151315
1,1,32,2010-02-05,False,42.31,2.572,211.096358,8.106,A,151315
2,1,59,2010-02-05,False,42.31,2.572,211.096358,8.106,A,151315
3,1,95,2010-02-05,False,42.31,2.572,211.096358,8.106,A,151315
4,2,8,2010-02-05,False,40.19,2.572,210.752605,8.324,A,202307
...,...,...,...,...,...,...,...,...,...,...
8239,44,5,2011-11-25,True,38.89,3.445,129.836400,6.078,C,39910
8240,44,95,2011-11-25,True,38.89,3.445,129.836400,6.078,C,39910
8241,45,27,2011-11-25,True,48.71,3.492,188.350400,8.523,B,118221
8242,45,36,2011-11-25,True,48.71,3.492,188.350400,8.523,B,118221


Next, we transform the boolean `IsHoliday` column into an integer format (`1` for True, `0` for False), which is required by most modeling libraries.

In [15]:
from sklearn.preprocessing import FunctionTransformer

# This custom transformer finds all boolean columns and converts them to integers.
bool_to_int = FunctionTransformer(
    lambda df: df.assign(
        **{col: df[col].astype(int) for col in df.select_dtypes(include='bool').columns}
    )
)

print("Value counts in IsHoliday column before transformation:", X_train_t['IsHoliday'].value_counts())

X_train_t = bool_to_int.fit_transform(X_train_t)
X_valid_t = bool_to_int.transform(X_valid_t)

print("Values counts in IsHoliday column after transformation", X_train_t['IsHoliday'].value_counts())


X_train_t

Value counts in IsHoliday column before transformation: IsHoliday
False    7633
True      611
Name: count, dtype: int64
Values counts in IsHoliday column after transformation IsHoliday
0    7633
1     611
Name: count, dtype: int64


,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Type,Size
0,1,19,2010-02-05,0,42.31,2.572,211.096358,8.106,A,151315
1,1,32,2010-02-05,0,42.31,2.572,211.096358,8.106,A,151315
2,1,59,2010-02-05,0,42.31,2.572,211.096358,8.106,A,151315
3,1,95,2010-02-05,0,42.31,2.572,211.096358,8.106,A,151315
4,2,8,2010-02-05,0,40.19,2.572,210.752605,8.324,A,202307
...,...,...,...,...,...,...,...,...,...,...
8239,44,5,2011-11-25,1,38.89,3.445,129.836400,6.078,C,39910
8240,44,95,2011-11-25,1,38.89,3.445,129.836400,6.078,C,39910
8241,45,27,2011-11-25,1,48.71,3.492,188.350400,8.523,B,118221
8242,45,36,2011-11-25,1,48.71,3.492,188.350400,8.523,B,118221


The `Type` column contains three distinct store types ('A', 'B', 'C'). We apply one-hot encoding to convert this categorical feature into three binary indicator columns. This allows linear models to interpret the store types as independent features.

In [16]:
set(X_train_t['Type'].values)

{'A', 'B', 'C'}

In [17]:
class CustomOneHotEncoder(BaseEstimator, TransformerMixin):
    """A custom one-hot encoder that is robust to missing categories in new data."""
    def __init__(self, columns=None):
        self.columns = columns
        self.ohe_columns_encodings = {}

    def fit(self, X, y=None):
        if self.columns is None:
            self.columns = X.select_dtypes(include=['object', 'category']).columns.tolist()
        # Learn the unique categories for each column from the training data
        for col in self.columns:
            unique_vals = X[col].dropna().unique()
            self.ohe_columns_encodings[col] = [f"{col}_{val}" for val in unique_vals]
        return self

    def transform(self, X):
        X_transformed = X.copy()
        for col in self.columns:
            # Create dummy variables for the current data
            dummies = pd.get_dummies(
                X_transformed[col],
                prefix=col,
                dummy_na=True, # Handle potential NaN values
                dtype=int
            )
            # Ensure all columns learned during fit are present
            for expected_col in self.ohe_columns_encodings[col]:
                if expected_col not in dummies.columns:
                    dummies[expected_col] = 0
            # Reorder columns to match the fitted schema
            dummies = dummies[self.ohe_columns_encodings[col]]
            # Concatenate new dummy columns and drop the original
            X_transformed = pd.concat([
                X_transformed.drop(col, axis=1),
                dummies
            ], axis=1)
        return X_transformed

In [18]:
one_hot_encoder = CustomOneHotEncoder(columns=['Type'])
X_train_t = one_hot_encoder.fit_transform(X_train_t)
X_valid_t = one_hot_encoder.transform(X_valid_t)

X_train_t

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Size,Type_A,Type_B,Type_C
0,1,19,2010-02-05,0,42.31,2.572,211.096358,8.106,151315,1,0,0
1,1,32,2010-02-05,0,42.31,2.572,211.096358,8.106,151315,1,0,0
2,1,59,2010-02-05,0,42.31,2.572,211.096358,8.106,151315,1,0,0
3,1,95,2010-02-05,0,42.31,2.572,211.096358,8.106,151315,1,0,0
4,2,8,2010-02-05,0,40.19,2.572,210.752605,8.324,202307,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8239,44,5,2011-11-25,1,38.89,3.445,129.836400,6.078,39910,0,0,1
8240,44,95,2011-11-25,1,38.89,3.445,129.836400,6.078,39910,0,0,1
8241,45,27,2011-11-25,1,48.71,3.492,188.350400,8.523,118221,0,1,0
8242,45,36,2011-11-25,1,48.71,3.492,188.350400,8.523,118221,0,1,0


To help the model capture temporal patterns, we engineer a rich set of time-based features from the `Date` column using a custom `FeatureAdder` transformer. These features include:

- **Date Components**: `WeekOfYear`, `Month`, `Year`.
- **Cyclical Features**: Sine and cosine transformations of the week and month to model seasonal patterns smoothly.
- **Holiday Proximity**: Features indicating the number of days until the next holiday and since the last holiday.


In [19]:
HOLIDAY_DATES = {
    # Super Bowl
    '2010-02-12': 'SuperBowl', '2011-02-11': 'SuperBowl', '2012-02-10': 'SuperBowl', '2013-02-08': 'SuperBowl',
    # Labor Day
    '2010-09-10': 'LaborDay', '2011-09-09': 'LaborDay', '2012-09-07': 'LaborDay', '2013-09-06': 'LaborDay',
    # Thanksgiving
    '2010-11-26': 'Thanksgiving', '2011-11-25': 'Thanksgiving', '2012-11-23': 'Thanksgiving', '2013-11-29': 'Thanksgiving',
    # Christmas
    '2010-12-31': 'Christmas', '2011-12-30': 'Christmas', '2012-12-28': 'Christmas', '2013-12-27': 'Christmas',
}

class FeatureAdder(BaseEstimator, TransformerMixin):
    """A comprehensive transformer for adding time-based and holiday-related features."""
    def __init__(self,
                 add_week_num=True,
                 add_holiday_flags=True,
                 add_holiday_proximity=True,
                 add_holiday_windows=False,
                 add_fourier_features=True,
                 add_month_and_year=True,
                 list_of_holiday_proximity=list(set(HOLIDAY_DATES.values())),
                 holiday_dates=HOLIDAY_DATES,
                 replace_time_index = True,
                 add_dummy_date = False,
                 start_date = None
                 ):

        self.holiday_dates = holiday_dates
        self.add_week_num = add_week_num
        self.add_month_and_year = add_month_and_year
        self.add_holiday_flags = add_holiday_flags
        self.add_holiday_proximity = add_holiday_proximity
        self.add_holiday_windows = add_holiday_windows
        self.add_fourier_features = add_fourier_features
        self.list_of_holiday_proximity = list_of_holiday_proximity
        self.replace_time_index = replace_time_index
        self.add_dummy_date = add_dummy_date
        self.start_date = start_date

    def fit(self, X, y=None):
        if self.start_date is not None:
          self.start_date_ = self.start_date
        else:
          self.start_date_ = pd.to_datetime(X['Date']).min()
        return self

    def transform(self, X):
        X_ = X.copy()
        X_['Date'] = pd.to_datetime(X_['Date'])

        if self.add_dummy_date:
          X_['DateDummy'] = ((X_['Date'] - self.start_date_).dt.days // 7).astype(int)

        if self.add_month_and_year or self.add_fourier_features:
          self._add_month_and_year(X_)

        if self.add_week_num:
            self._add_week_number(X_)

        if self.add_holiday_flags:
            self._add_specific_holiday_flags(X_)

        if self.add_holiday_proximity:
            self._add_proximity_to_holidays(X_)

        if self.add_holiday_windows:
            self._add_pre_post_holiday_windows(X_)

        if self.add_fourier_features and 'WeekOfYear' in X_.columns:
            self._add_fourier_features(X_)

        if self.list_of_holiday_proximity:
            self._add_proximity_to_specific_holidays(X_)

        if self.replace_time_index:
          self._replace_date_with_time_index(X_)

        return X_

    def _add_week_number(self, df):
        df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)

    def _add_month_and_year(self, df):
        df['Month'] = df['Date'].dt.month
        df['Year'] = df['Date'].dt.year

    def _add_specific_holiday_flags(self, df):
      date_str = df['Date'].dt.strftime('%Y-%m-%d')

      for holiday_name in set(self.holiday_dates.values()):
          holiday_dates = {
              date for date, name in self.holiday_dates.items() if name == holiday_name
          }
          df[f'Is_{holiday_name}'] = date_str.isin(holiday_dates).astype(int)

    def _replace_date_with_time_index(self, df):
        if self.start_date_ is None:
            raise RuntimeError("The transformer has not been fitted yet. Call .fit() before .transform().")
        dates = pd.to_datetime(df['Date'])
        time_delta_days = (dates - self.start_date_).dt.days
        df.drop(columns=['Date'], inplace=True)
        df['Date'] = (time_delta_days / 7).astype(int)

    def _add_proximity_to_holidays(self, df):
      holiday_dates = sorted([pd.to_datetime(d) for d in self.holiday_dates.keys()])
      safe_dates = pd.to_datetime(df['Date'], errors='coerce')
      indices = np.searchsorted(holiday_dates, safe_dates)

      next_holiday_dates = [holiday_dates[i] if i < len(holiday_dates) else pd.NaT for i in indices]
      df['Days_until_next_holiday'] = (pd.to_datetime(next_holiday_dates) - df['Date']).dt.days

      last_holiday_dates = [holiday_dates[i-1] if i > 0 else pd.NaT for i in indices]
      df['Days_since_last_holiday'] = (df['Date'] - pd.to_datetime(last_holiday_dates)).dt.days

      df.fillna({'Days_until_next_holiday': 999, 'Days_since_last_holiday': 999}, inplace=True)

    def _add_proximity_to_specific_holidays(self, df):
      safe_dates = pd.to_datetime(df['Date'], errors='coerce')
      for holiday in self.list_of_holiday_proximity:
        holiday_dates = sorted([pd.to_datetime(d) for d, name in self.holiday_dates.items() if name == holiday])
        if len(holiday_dates) == 0:
          continue
        indices = np.searchsorted(holiday_dates, safe_dates)

        next_holiday_dates = [holiday_dates[i] if i < len(holiday_dates) else pd.NaT for i in indices]
        df[f'Days_until_next_{holiday}'] = (pd.to_datetime(next_holiday_dates) - df['Date']).dt.days

        last_holiday_dates = [holiday_dates[i-1] if i > 0 else pd.NaT for i in indices]
        df[f'Days_since_last_{holiday}'] = (df['Date'] - pd.to_datetime(last_holiday_dates)).dt.days

        df.fillna({f'Days_until_next_{holiday}': 999, f'Days_since_last_{holiday}': 999}, inplace=True)

    def _add_pre_post_holiday_windows(self, df):
      unique_holidays = set(self.holiday_dates.values())
      for holiday_name in unique_holidays:
          holiday_specific_dates = pd.to_datetime([
              date_str for date_str, name in self.holiday_dates.items() if name == holiday_name
          ])

          df[f'Is_7_Days_Before_{holiday_name}'] = 0
          df[f'Is_7_Days_After_{holiday_name}'] = 0

          for holiday_date in holiday_specific_dates:
              before_mask = (df['Date'] >= holiday_date - pd.Timedelta(days=7)) & (df['Date'] < holiday_date)
              after_mask = (df['Date'] > holiday_date) & (df['Date'] <= holiday_date + pd.Timedelta(days=7))
              df.loc[before_mask, f'Is_7_Days_Before_{holiday_name}'] = 1
              df.loc[after_mask, f'Is_7_Days_After_{holiday_name}'] = 1

    def _add_fourier_features(self, df):
        """Adds sine and cosine features for week and month to capture cyclicity."""
        df['week_sin'] = np.sin(2 * np.pi * df['WeekOfYear'] / 52)
        df['week_cos'] = np.cos(2 * np.pi * df['WeekOfYear'] / 52)
        df['month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)

In [20]:
params = {
    'add_week_num' : True,
    'add_holiday_flags' : False,
    'add_holiday_proximity': True,
    'add_holiday_windows': False,
    'add_fourier_features': True,
    'add_month_and_year': True,
    'replace_time_index': False,
    'list_of_holiday_proximity': [],
}

feature_adder = FeatureAdder(**params)
X_train_t = feature_adder.fit_transform(X_train_t)
X_valid_t = feature_adder.transform(X_valid_t)

X_train_t

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Size,Type_A,...,Type_C,Month,Year,WeekOfYear,Days_until_next_holiday,Days_since_last_holiday,week_sin,week_cos,month_sin,month_cos
0,1,19,2010-02-05,0,42.31,2.572,211.096358,8.106,151315,1,...,0,2,2010,5,7,999.0,0.568065,0.822984,0.866025,0.500000
1,1,32,2010-02-05,0,42.31,2.572,211.096358,8.106,151315,1,...,0,2,2010,5,7,999.0,0.568065,0.822984,0.866025,0.500000
2,1,59,2010-02-05,0,42.31,2.572,211.096358,8.106,151315,1,...,0,2,2010,5,7,999.0,0.568065,0.822984,0.866025,0.500000
3,1,95,2010-02-05,0,42.31,2.572,211.096358,8.106,151315,1,...,0,2,2010,5,7,999.0,0.568065,0.822984,0.866025,0.500000
4,2,8,2010-02-05,0,40.19,2.572,210.752605,8.324,202307,1,...,0,2,2010,5,7,999.0,0.568065,0.822984,0.866025,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8239,44,5,2011-11-25,1,38.89,3.445,129.836400,6.078,39910,0,...,1,11,2011,47,0,77.0,-0.568065,0.822984,-0.500000,0.866025
8240,44,95,2011-11-25,1,38.89,3.445,129.836400,6.078,39910,0,...,1,11,2011,47,0,77.0,-0.568065,0.822984,-0.500000,0.866025
8241,45,27,2011-11-25,1,48.71,3.492,188.350400,8.523,118221,0,...,0,11,2011,47,0,77.0,-0.568065,0.822984,-0.500000,0.866025
8242,45,36,2011-11-25,1,48.71,3.492,188.350400,8.523,118221,0,...,0,11,2011,47,0,77.0,-0.568065,0.822984,-0.500000,0.866025


In [21]:
class BoolToInt(BaseEstimator, TransformerMixin):
    """A simple transformer to convert all boolean columns in a DataFrame to integers."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_ = X.copy()
        for col in X_.select_dtypes(include='bool').columns:
            X_[col] = X_[col].astype(int)
        return X_

Finally, we assemble all the preprocessing steps into a single `scikit-learn` Pipeline. This ensures that the same transformations are consistently applied to both the training and validation data.

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

columns_to_drop = [
    'MarkDown1', 'MarkDown2', 'MarkDown3',
    'MarkDown4', 'MarkDown5'
]

time_feature_params = {
    'add_week_num': True,
    'add_holiday_flags': False,
    'add_holiday_proximity': True,
    'add_holiday_windows': False,
    'add_fourier_features': True,
    'add_month_and_year': True,
    'replace_time_index': False,
    'list_of_holiday_proximity': [],
}

# Define the complete preprocessing pipeline
preprocess = Pipeline(steps=[
    ('drop_markdown', ChangeColumns(columns_to_drop=columns_to_drop)),
    ('bool_to_int', BoolToInt()),
    ('type_encoding', CustomOneHotEncoder(columns=['Type'])),
    ('add_time_features', FeatureAdder(**time_feature_params)),
])

# Apply the pipeline to the training and validation data
X_train_t = preprocess.fit_transform(X_train)
X_valid_t = preprocess.transform(X_valid)

X_train_t

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Size,Type_A,...,Type_C,Month,Year,WeekOfYear,Days_until_next_holiday,Days_since_last_holiday,week_sin,week_cos,month_sin,month_cos
0,1,19,2010-02-05,0,42.31,2.572,211.096358,8.106,151315,1,...,0,2,2010,5,7,999.0,0.568065,0.822984,0.866025,0.500000
1,1,32,2010-02-05,0,42.31,2.572,211.096358,8.106,151315,1,...,0,2,2010,5,7,999.0,0.568065,0.822984,0.866025,0.500000
2,1,59,2010-02-05,0,42.31,2.572,211.096358,8.106,151315,1,...,0,2,2010,5,7,999.0,0.568065,0.822984,0.866025,0.500000
3,1,95,2010-02-05,0,42.31,2.572,211.096358,8.106,151315,1,...,0,2,2010,5,7,999.0,0.568065,0.822984,0.866025,0.500000
4,2,8,2010-02-05,0,40.19,2.572,210.752605,8.324,202307,1,...,0,2,2010,5,7,999.0,0.568065,0.822984,0.866025,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8239,44,5,2011-11-25,1,38.89,3.445,129.836400,6.078,39910,0,...,1,11,2011,47,0,77.0,-0.568065,0.822984,-0.500000,0.866025
8240,44,95,2011-11-25,1,38.89,3.445,129.836400,6.078,39910,0,...,1,11,2011,47,0,77.0,-0.568065,0.822984,-0.500000,0.866025
8241,45,27,2011-11-25,1,48.71,3.492,188.350400,8.523,118221,0,...,0,11,2011,47,0,77.0,-0.568065,0.822984,-0.500000,0.866025
8242,45,36,2011-11-25,1,48.71,3.492,188.350400,8.523,118221,0,...,0,11,2011,47,0,77.0,-0.568065,0.822984,-0.500000,0.866025


# Model Training

With the data preprocessed, we can now train our forecasting model. We will use a custom-wrapped SARIMAX model that fits an independent time-series model for each store-department pair.

In [23]:
import numpy as np
from sklearn.metrics import mean_absolute_error

def compute_wmae(y_true, y_pred, is_holiday):
    """Computes the Weighted Mean Absolute Error (WMAE) as defined by the competition."""
    weights = np.where(np.array(is_holiday), 5, 1)
    return mean_absolute_error(y_true, y_pred, sample_weight=weights)

## ARIMA Model

We begin by training a set of **ARIMA** models as a baseline. ARIMA is a classical time-series forecasting method that models the next step in a sequence as a linear function of past observations and errors.

Our custom `StoreDeptSARIMAX` class fits a separate SARIMAX model for each `(Store, Dept)` combination. For this initial baseline, we configure the model to be a simple ARIMA `(1,1,1)` and do **not** use any of the exogenous features we created. This allows us to measure the predictive power of the time-series component alone.

In [24]:
from sklearn.base import BaseEstimator, RegressorMixin
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.exceptions import NotFittedError

class StoreDeptSARIMAX(BaseEstimator, RegressorMixin):
    """A scikit-learn compatible wrapper for fitting a SARIMAX model to each store-department group."""
    def __init__(self, order=(1, 1, 1), seasonal_order=(0, 0, 0, 0),
                 use_all_exog=False):
        self.order = order
        self.seasonal_order = seasonal_order
        self.use_all_exog = use_all_exog
        self.exog_cols = None
        self.models_ = {}

    def fit(self, X, y):
        """
        Fits a separate SARIMAX model for each (Store, Dept) group.
        Suppresses convergence and other warnings for smoother execution.
        """
        data = X.copy()
        data["y"] = y

        if self.use_all_exog:
            exclude_cols = {"Store", "Dept", "Date", "y"}
            self.exog_cols = [col for col in data.columns if col not in exclude_cols]
            if not self.exog_cols: self.exog_cols = None

        for key, group in data.groupby(["Store", "Dept"]):
            group = group.sort_values("Date")
            group['Date'] = pd.to_datetime(group['Date'])
            group = group.set_index('Date')

            ts = group['y']
            if ts.dropna().empty: continue

            exog_data = group[self.exog_cols] if self.exog_cols else None

            # Suppress warnings during model initialization and fitting
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore")
                try:
                    model = SARIMAX(
                        endog=ts, exog=exog_data, order=self.order,
                        seasonal_order=self.seasonal_order,
                        enforce_stationarity=False, enforce_invertibility=False
                    )
                    fitted_model = model.fit(disp=False)
                    self.models_[key] = fitted_model
                except Exception:
                    # Silently skip any group that fails to fit
                    pass
        return self

    def predict(self, X):
        """
        Generates predictions for each row in X using the corresponding fitted model.
        Fills with NaN if a model for a group does not exist or prediction fails.
        """
        if not hasattr(self, 'models_') or not self.models_:
            raise NotFittedError("This model has not been fitted yet. Call fit() before predict().")

        X_pred = X.copy()
        X_pred['Date'] = pd.to_datetime(X_pred['Date'])

        predictions = pd.Series(index=X_pred.index, dtype=float)

        for key, group in X_pred.groupby(["Store", "Dept"]):
            if key in self.models_:
                with warnings.catch_warnings():
                    warnings.filterwarnings("ignore")
                    try:
                        model = self.models_[key]
                        group_for_pred = group.drop_duplicates(subset=['Date']).sort_values("Date").set_index("Date")

                        if group_for_pred.empty: continue

                        start_date = group_for_pred.index.min()
                        end_date = group_for_pred.index.max()
                        exog_data = group_for_pred[self.exog_cols] if self.exog_cols else None

                        group_preds_by_date = model.predict(
                            start=start_date, end=end_date, exog=exog_data
                        )

                        # Map predictions back to the original index to handle duplicated dates
                        mapped_preds = group['Date'].map(group_preds_by_date)
                        mapped_preds.index = group.index

                        predictions.update(mapped_preds)
                    except Exception:
                        # If prediction fails, values remain NaN
                        pass

        return predictions

In [25]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Create a minimal feature set for the baseline ARIMA model
X_train_T = X_train_t[['Date', 'Store', 'Dept']]
X_valid_T = X_valid_t[['Date', 'Store', 'Dept']]

arima_model1 = StoreDeptSARIMAX(
    order=(1, 1, 1), # (p, d, q)
    use_all_exog=False, # Do not use any external features
)

arima_model1.fit(X_train_T, y_train)

# Generate and evaluate predictions
train_preds = arima_model1.predict(X_train_T).fillna(0) # Fill NaNs for failed predictions
valid_preds = arima_model1.predict(X_valid_T).fillna(0)

train_wmae = compute_wmae(y_train, train_preds, is_holiday=X_train_t['IsHoliday'])
valid_wmae = compute_wmae(y_valid, valid_preds, is_holiday=X_valid_t['IsHoliday'])
train_mae = mean_absolute_error(y_train, train_preds)
valid_mae = mean_absolute_error(y_valid, valid_preds)

print(f"Train WMAE: {train_wmae:.2f}, MAE: {train_mae:.2f}")
print(f"Valid WMAE: {valid_wmae:.2f}, MAE: {valid_mae:.2f}")

Train WMAE: 16385.21, MAE: 16150.83
Valid WMAE: 17585.58, MAE: 16558.30


In [26]:
# Configuration for logging and reproducibility
TIME_FEATURE_PRESETS = {
    "default": [
        'Month', 'Year', 'WeekOfYear', 'HolidayFlags',
        'week_sin', 'week_cos', 'month_sin', 'month_cos',
        'Days_until_next_holiday', 'Days_since_next_holiday',
        "Is_7_Days_Before_some_holiday", "Is_7_Days_After_Some_Holiday",
    ],
    "minimal": ['Month', 'Year'],
}

cfg = {
    'merge1': 'train, store, how=left, on=Store',
    'merge2': 'train, features, how=left, on=Store, Date, IsHoliday',
    'merged_tables': ['train', 'stores', 'features'],
    'time_features': TIME_FEATURE_PRESETS['default'],
    'add_dummy_date': False,
    'replace_time_index': True,
    'start_date': '2010-02-05',
    'score_metric': 'WMAE',
    'score_policy': {
        'weight on holidays': 5,
        'weight on non_holidays': 1
    },
}

arima_config = {
    'order': (1, 1, 1),
    'use_all_exog': False,
    'filterwarnings': True,
    'verbose': 0
}

In [27]:
import wandb
import joblib

def log_to_wandb(model, train_score, val_score, config,
                       run_name="run_00",
                       project_name="Walmart Recruiting - Store Sales Forecasting",
                       artifact_name="pipeline",
                       artifact_type="model",
                       artifact_description=""):
    """Logs model, scores, and configuration to Weights & Biases."""
    filename = f"{run_name}.pkl"
    joblib.dump(model, filename)

    wandb.init(project=project_name, name=run_name, config=config)

    wandb.log({
        'train_wmae': train_score,
        'val_wmae': val_score
    })

    artifact = wandb.Artifact(
        name=artifact_name,
        type=artifact_type,
        description=artifact_description
    )
    artifact.add_file(filename)
    wandb.log_artifact(artifact)

    wandb.finish()

In [28]:
from sklearn.pipeline import Pipeline

# The final model is a pipeline containing preprocessing and the SARIMAX wrapper
pipeline = Pipeline([
    ('preprocess', preprocess),
    ('model', arima_model1)
])

# Create a merged configuration for logging
cur_config = (cfg.copy())
cur_config['replace_time_index'] = False
# The 'HolidayFlags' preset was not used in the FeatureAdder config
if 'HolidayFlags' in cur_config['time_features']:
    cur_config['time_features'].remove('HolidayFlags')
merged_config = {**cur_config, **arima_config}


log_to_wandb(
    model=pipeline,
    train_score=train_wmae,
    val_score=valid_wmae,
    config=merged_config,
    run_name='arima_baseline_01',
    artifact_name="arima-baseline",
    artifact_description="Baseline ARIMA(1,1,1) model without exogenous features."
)

wandb: Currently logged in as: lchik22 (lchik22-free-uni) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


train_wmae,▁
val_wmae,▁
train_wmae,16385.21391
val_wmae,17585.58415


### Conclusion

The simple ARIMA(1,1,1) model, which relies solely on the temporal structure of the data, achieved a validation **WMAE of approximately 17,585**.

This result serves as a crucial baseline. While the error is high, it confirms that there is a predictable time-series signal within the data. However, the magnitude of the error clearly indicates that autoregressive components alone are insufficient for accurate sales forecasting. This underscores the need to incorporate the exogenous features (like CPI, Fuel Price, and holiday information) that we engineered earlier to build more powerful and accurate models.